<a href="https://colab.research.google.com/github/tusharvatsa32/VisTransformers/blob/main/Code/Copy_of_VisionTransformer_ViT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu Apr 29 16:08:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# License: BSD
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

In [3]:
!pip install vit-pytorch

In [4]:
import torch
from vit_pytorch import ViT

v = ViT(
    image_size = 256,
    patch_size = 32,
    num_classes = 10,
    dim = 1024,
    depth = 6,
    heads = 16,
    mlp_dim = 2048,
    dropout = 0.3,
    emb_dropout = 0.1
)

img = torch.randn(1, 3, 256, 256)

preds = v(img) # (1, 1000)

In [7]:
# Importing Libraries
import torch                       # pytorch
import torch.nn as nn              # pytorch for neural network
import numpy as np                 # for algebric functions
import matplotlib.pyplot as plt    # to plot graph
import PIL


# torch vision package
import torchvision                 # for handling image & has CNN architecture
from torchvision import transforms

In [8]:
import PIL
# img_size = ((224, 224)) For ResNet models
img_size = ((256, 256)) # For ViT predefined weights

transforms_train = transforms.Compose([
    transforms.Resize(img_size),
    transforms.ColorJitter(hue=.05, saturation=.05),
    transforms.RandomHorizontalFlip(p=0.3),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.RandomRotation(10, resample=PIL.Image.BILINEAR),
    transforms.ToTensor(),
    transforms.RandomCrop(img_size, fill=0),
    transforms.RandomAffine(10, translate=None, scale=(0.8, 1.2), shear=None, fill=0, fillcolor=None, resample=None),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    #transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transforms_val = transforms.Compose([
    transforms.Resize(img_size),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    #transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:1201: UserWarning: Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead
  "Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead"


In [13]:
#transform --> transform the data during creation (ToTensor())
#download  --> to download to local file
#root      --> data storage place
#train     --> means training data from training set 
#type(trainset)
trainset = torchvision.datasets.CIFAR10(train=True,download=True,root= "./cifar10/train_data", transform=transforms_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=256,
                                          shuffle=True, num_workers=8)

testset = torchvision.datasets.CIFAR10(train=False,download=True,root= "./cifar10/test_data", transform=transforms_val)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                         shuffle=False, num_workers=8)

Files already downloaded and verified


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Files already downloaded and verified


In [10]:
trainset.classes

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [11]:
#class labels [there are 10 lables]
#this is the order of lable of this dataset
classes = trainset.classes
print(classes)

['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [12]:
numEpochs = 100
in_features = 3 # RGB channels

learningRate = 0.03
weightDecay = 5e-5

num_classes = len(trainset.classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = ViT(
    image_size = 256,
    patch_size = 32,
    num_classes = 10,
    dim = 1024,
    depth = 6,
    heads = 16,
    mlp_dim = 2048,
    dropout = 0.3,
    emb_dropout = 0.1
)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9, nesterov=True)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.3, patience=3, threshold=0.002, verbose=True)

In [15]:
model

ViT(
  (to_patch_embedding): Sequential(
    (0): Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=32, p2=32)
    (1): Linear(in_features=3072, out_features=1024, bias=True)
  )
  (dropout): Dropout(p=0.1, inplace=False)
  (transformer): Transformer(
    (layers): ModuleList(
      (0): ModuleList(
        (0): PreNorm(
          (norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fn): Attention(
            (attend): Softmax(dim=-1)
            (to_qkv): Linear(in_features=1024, out_features=3072, bias=False)
            (to_out): Sequential(
              (0): Linear(in_features=1024, out_features=1024, bias=True)
              (1): Dropout(p=0.3, inplace=False)
            )
          )
        )
        (1): PreNorm(
          (norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fn): FeedForward(
            (net): Sequential(
              (0): Linear(in_features=1024, out_features=2048, bias=True)
              (1): GELU()
        

In [ ]:
model

VisionTransformer(
  (embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (transformer): Encoder(
    (pos_embedding): PositionEmbs(
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder_layers): ModuleList(
      (0): EncoderBlock(
        (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): SelfAttention(
          (query): LinearGeneral()
          (key): LinearGeneral()
          (value): LinearGeneral()
          (out): LinearGeneral()
        )
        (dropout): Dropout(p=0.1, inplace=False)
        (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MlpBlock(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (act): GELU()
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
      (1): EncoderBlock(
        (norm1): LayerNorm((768,)

In [ ]:
# model = models.resnet101(pretrained=True)

In [ ]:
for param in model.parameters():
  print(param.requires_grad)

In [ ]:
# for param in model_ft.parameters():
#     param.requires_grad = False

In [ ]:
model = models.resnet101(pretrained=True)

# for param in model_ft.parameters():
#     param.requires_grad = False

  
num_ftrs = model.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).

num_classes = len(trainset.classes)

model.fc = nn.Linear(num_ftrs, num_classes)

learningRate = 0.01
weightDecay = 5e-5

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9, nesterov=True)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3, threshold=0.002, verbose=True)

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/hub/checkpoints/resnet101-5d3b4d8f.pth


In [17]:
my_acc = []
my_loss = []

In [ ]:
# Train!
numEpochs = 100
for epoch in range(numEpochs):
    
    # Train
    model.train()
    train_loss = 0.0
    train_acc = 0.0
    correct = 0

    for batch_num, (x, y) in enumerate(trainloader):
        optimizer.zero_grad()
        
        x, y = x.to(device), y.to(device)

        outputs = model(x)

        correct += (torch.argmax(outputs, axis=1) == y).sum().item()

        loss = criterion(outputs, y.long())
        loss.backward()
        optimizer.step()

        del(outputs)

        train_loss += loss.item()

        if batch_num % 100 == 0:
            print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch, batch_num+1, train_loss/(batch_num+1)))

    train_accuracy = correct / len(trainset)

    # Validate
    model.eval()
    num_correct = 0
    for batch_num1, (x, y) in enumerate(testloader):
        x, y = x.to(device), y.to(device)
        outputs = model(x)

        num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()

    val_accuracy = num_correct / len(testset)
    my_acc.append(val_accuracy)
    my_loss.append(train_loss/(batch_num+1))
    print('Epoch: {}\t Training Accuracy: {:.4f}\t Validation Accuracy: {:.4f}\t Avg-Loss: {:.4f}'.format(epoch, train_accuracy*100, val_accuracy * 100, train_loss/(batch_num+1)))
    scheduler.step(val_accuracy)

    #torch.save(network.state_dict(),'/content/drive/MyDrive/DL_CMU/HW2_P2/ResNet_Plateau_d3/Net_'+str(epoch)+'_'+str(val_accuracy)+'_checkpoint.t7')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch: 0	Batch: 1	Avg-Loss: 2.3813
Epoch: 0	Batch: 101	Avg-Loss: 7.5015
Epoch: 0	 Training Accuracy: 16.3000	 Validation Accuracy: 16.6400	 Avg-Loss: 4.9104
Epoch: 1	Batch: 1	Avg-Loss: 2.2053
Epoch: 1	Batch: 101	Avg-Loss: 2.1176
Epoch: 1	 Training Accuracy: 21.1780	 Validation Accuracy: 24.0600	 Avg-Loss: 2.0735
Epoch: 2	Batch: 1	Avg-Loss: 2.0606
Epoch: 2	Batch: 101	Avg-Loss: 1.9741
Epoch: 2	 Training Accuracy: 26.1400	 Validation Accuracy: 28.0700	 Avg-Loss: 1.9577
Epoch: 3	Batch: 1	Avg-Loss: 1.9816
Epoch: 3	Batch: 101	Avg-Loss: 1.9090
Epoch: 3	 Training Accuracy: 29.2040	 Validation Accuracy: 32.5600	 Avg-Loss: 1.8757
Epoch: 4	Batch: 1	Avg-Loss: 1.8631
Epoch: 4	Batch: 101	Avg-Loss: 1.8162
Epoch: 4	 Training Accuracy: 32.5520	 Validation Accuracy: 35.6400	 Avg-Loss: 1.7867
Epoch: 5	Batch: 1	Avg-Loss: 1.7688
Epoch: 5	Batch: 101	Avg-Loss: 1.6953
Epoch: 5	 Training Accuracy: 37.5220	 Validation Accuracy: 41.0800	 Avg-Loss: 1.6764
Epoch: 6	Batch: 1	Avg-Loss: 1.5842
Epoch: 6	Batch: 101	Avg

In [ ]:
# Train!
numEpochs = 100
for epoch in range(numEpochs):
    
    # Train
    model.train()
    train_loss = 0.0
    train_acc = 0.0
    correct = 0

    for batch_num, (x, y) in enumerate(trainloader):
        optimizer.zero_grad()
        
        x, y = x.to(device), y.to(device)

        outputs = model(x)

        correct += (torch.argmax(outputs, axis=1) == y).sum().item()

        loss = criterion(outputs, y.long())
        loss.backward()
        optimizer.step()

        del(outputs)

        train_loss += loss.item()

        if batch_num % 100 == 0:
            print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch, batch_num+1, train_loss/(batch_num+1)))

    train_accuracy = correct / len(trainset)

    # Validate
    model.eval()
    num_correct = 0
    for batch_num1, (x, y) in enumerate(testloader):
        x, y = x.to(device), y.to(device)
        outputs = model(x)

        num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()

    val_accuracy = num_correct / len(testset)
    my_acc.append(val_accuracy)
    my_loss.append(train_loss/(batch_num+1))
    print('Epoch: {}\t Training Accuracy: {:.4f}\t Validation Accuracy: {:.4f}\t Avg-Loss: {:.4f}'.format(epoch, train_accuracy*100, val_accuracy * 100, train_loss/(batch_num+1)))
    scheduler.step(val_accuracy)

    #torch.save(network.state_dict(),'/content/drive/MyDrive/DL_CMU/HW2_P2/ResNet_Plateau_d3/Net_'+str(epoch)+'_'+str(val_accuracy)+'_checkpoint.t7')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch: 0	Batch: 1	Avg-Loss: 2.2711
Epoch: 0	Batch: 101	Avg-Loss: 3.0298
Epoch: 0	Batch: 201	Avg-Loss: 2.6981
Epoch: 0	Batch: 301	Avg-Loss: 2.5748
Epoch: 0	Batch: 401	Avg-Loss: 2.4844
Epoch: 0	Batch: 501	Avg-Loss: 2.4158
Epoch: 0	Batch: 601	Avg-Loss: 2.3639
Epoch: 0	Batch: 701	Avg-Loss: 2.3241
Epoch: 0	Batch: 801	Avg-Loss: 2.2968
Epoch: 0	Batch: 901	Avg-Loss: 2.2637
Epoch: 0	Batch: 1001	Avg-Loss: 2.2473
Epoch: 0	Batch: 1101	Avg-Loss: 2.2248
Epoch: 0	Batch: 1201	Avg-Loss: 2.2034
Epoch: 0	Batch: 1301	Avg-Loss: 2.1900
Epoch: 0	Batch: 1401	Avg-Loss: 2.1761
Epoch: 0	Batch: 1501	Avg-Loss: 2.1634
Epoch: 0	Batch: 1601	Avg-Loss: 2.1485
Epoch: 0	Batch: 1701	Avg-Loss: 2.1341
Epoch: 0	Batch: 1801	Avg-Loss: 2.1234
Epoch: 0	Batch: 1901	Avg-Loss: 2.1115
Epoch: 0	Batch: 2001	Avg-Loss: 2.0997
Epoch: 0	Batch: 2101	Avg-Loss: 2.0867
Epoch: 0	Batch: 2201	Avg-Loss: 2.0780
Epoch: 0	Batch: 2301	Avg-Loss: 2.0682
Epoch: 0	Batch: 2401	Avg-Loss: 2.0593
Epoch: 0	Batch: 2501	Avg-Loss: 2.0514
Epoch: 0	Batch: 2601	Avg

RuntimeError: ignored

In [ ]:
# Train!
numEpochs = 100
for epoch in range(numEpochs):
    
    # Train
    model.train()
    train_loss = 0.0
    train_acc = 0.0
    correct = 0

    for batch_num, (x, y) in enumerate(trainloader):
        optimizer.zero_grad()
        
        x, y = x.to(device), y.to(device)

        outputs = model(x)

        correct += (torch.argmax(outputs, axis=1) == y).sum().item()

        loss = criterion(outputs, y.long())
        loss.backward()
        optimizer.step()

        del(outputs)

        train_loss += loss.item()

        if batch_num % 100 == 0:
            print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch, batch_num+1, train_loss/(batch_num+1)))

    train_accuracy = correct / len(trainset)

    # Validate
    model.eval()
    num_correct = 0
    for batch_num1, (x, y) in enumerate(testloader):
        x, y = x.to(device), y.to(device)
        outputs = model(x)

        num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()

    val_accuracy = num_correct / len(testset)
    my_acc.append(val_accuracy)
    my_loss.append(train_loss/(batch_num+1))
    print('Epoch: {}\t Training Accuracy: {:.4f}\t Validation Accuracy: {:.4f}\t Avg-Loss: {:.4f}'.format(epoch, train_accuracy*100, val_accuracy * 100, train_loss/(batch_num+1)))
    scheduler.step(val_accuracy)

    #torch.save(network.state_dict(),'/content/drive/MyDrive/DL_CMU/HW2_P2/ResNet_Plateau_d3/Net_'+str(epoch)+'_'+str(val_accuracy)+'_checkpoint.t7')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch: 0	Batch: 1	Avg-Loss: 4.5812
Epoch: 0	Batch: 101	Avg-Loss: 2.9234
Epoch: 0	Batch: 201	Avg-Loss: 2.3388
Epoch: 0	Batch: 301	Avg-Loss: 2.0906
Epoch: 0	Batch: 401	Avg-Loss: 1.9510
Epoch: 0	Batch: 501	Avg-Loss: 1.8462
Epoch: 0	Batch: 601	Avg-Loss: 1.7737
Epoch: 0	Batch: 701	Avg-Loss: 1.7129
Epoch: 0	 Training Accuracy: 53.8980	 Validation Accuracy: 66.3800	 Avg-Loss: 1.6778
Epoch: 1	Batch: 1	Avg-Loss: 1.2406
Epoch: 1	Batch: 101	Avg-Loss: 1.2045
Epoch: 1	Batch: 201	Avg-Loss: 1.1665
Epoch: 1	Batch: 301	Avg-Loss: 1.1613
Epoch: 1	Batch: 401	Avg-Loss: 1.1522
Epoch: 1	Batch: 501	Avg-Loss: 1.1423
Epoch: 1	Batch: 601	Avg-Loss: 1.1416
Epoch: 1	Batch: 701	Avg-Loss: 1.1299
Epoch: 1	 Training Accuracy: 67.1060	 Validation Accuracy: 69.4100	 Avg-Loss: 1.1280
Epoch: 2	Batch: 1	Avg-Loss: 1.3802
Epoch: 2	Batch: 101	Avg-Loss: 0.9010
Epoch: 2	Batch: 201	Avg-Loss: 0.9174
Epoch: 2	Batch: 301	Avg-Loss: 0.9298
Epoch: 2	Batch: 401	Avg-Loss: 0.9250
Epoch: 2	Batch: 501	Avg-Loss: 0.9297
Epoch: 2	Batch: 601	Av

KeyboardInterrupt: ignored

In [ ]:
# Train!
numEpochs = 100
for epoch in range(numEpochs):
    
    # Train
    model.train()
    train_loss = 0.0
    train_acc = 0.0
    correct = 0

    for batch_num, (x, y) in enumerate(trainloader):
        optimizer.zero_grad()
        
        x, y = x.to(device), y.to(device)

        outputs = model(x)

        correct += (torch.argmax(outputs, axis=1) == y).sum().item()

        loss = criterion(outputs, y.long())
        loss.backward()
        optimizer.step()

        del(outputs)

        train_loss += loss.item()

        if batch_num % 100 == 0:
            print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch, batch_num+1, train_loss/(batch_num+1)))

    train_accuracy = correct / len(trainset)

    # Validate
    model.eval()
    num_correct = 0
    for batch_num1, (x, y) in enumerate(testloader):
        x, y = x.to(device), y.to(device)
        outputs = model(x)

        num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()

    val_accuracy = num_correct / len(testset)
    my_acc.append(val_accuracy)
    my_loss.append(train_loss/(batch_num+1))
    print('Epoch: {}\t Training Accuracy: {:.4f}\t Validation Accuracy: {:.4f}\t Avg-Loss: {:.4f}'.format(epoch, train_accuracy*100, val_accuracy * 100, train_loss/(batch_num+1)))
    scheduler.step(val_accuracy)

    #torch.save(network.state_dict(),'/content/drive/MyDrive/DL_CMU/HW2_P2/ResNet_Plateau_d3/Net_'+str(epoch)+'_'+str(val_accuracy)+'_checkpoint.t7')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch: 0	Batch: 1	Avg-Loss: 2.3129
Epoch: 0	Batch: 101	Avg-Loss: 0.9381
Epoch: 0	Batch: 201	Avg-Loss: 0.7681
Epoch: 0	Batch: 301	Avg-Loss: 0.6827
Epoch: 0	Batch: 401	Avg-Loss: 0.6277
Epoch: 0	Batch: 501	Avg-Loss: 0.5897
Epoch: 0	Batch: 601	Avg-Loss: 0.5605
Epoch: 0	Batch: 701	Avg-Loss: 0.5337
Epoch: 0	 Training Accuracy: 82.4640	 Validation Accuracy: 90.7100	 Avg-Loss: 0.5125
Epoch: 1	Batch: 1	Avg-Loss: 0.3835
Epoch: 1	Batch: 101	Avg-Loss: 0.3248
Epoch: 1	Batch: 201	Avg-Loss: 0.3260
Epoch: 1	Batch: 301	Avg-Loss: 0.3207
Epoch: 1	Batch: 401	Avg-Loss: 0.3191
Epoch: 1	Batch: 501	Avg-Loss: 0.3156
Epoch: 1	Batch: 601	Avg-Loss: 0.3126
Epoch: 1	Batch: 701	Avg-Loss: 0.3103
Epoch: 1	 Training Accuracy: 89.3980	 Validation Accuracy: 90.8000	 Avg-Loss: 0.3056
Epoch: 2	Batch: 1	Avg-Loss: 0.2628
Epoch: 2	Batch: 101	Avg-Loss: 0.2328
Epoch: 2	Batch: 201	Avg-Loss: 0.2375
Epoch: 2	Batch: 301	Avg-Loss: 0.2359
Epoch: 2	Batch: 401	Avg-Loss: 0.2337
Epoch: 2	Batch: 501	Avg-Loss: 0.2338
Epoch: 2	Batch: 601	Av

In [ ]:
# Train!
numEpochs = 100
for epoch in range(numEpochs):
    
    # Train
    model.train()
    train_loss = 0.0
    train_acc = 0.0
    correct = 0

    for batch_num, (x, y) in enumerate(trainloader):
        optimizer.zero_grad()
        
        x, y = x.to(device), y.to(device)

        outputs = model(x)

        correct += (torch.argmax(outputs, axis=1) == y).sum().item()

        loss = criterion(outputs, y.long())
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        if batch_num % 100 == 0:
            print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch, batch_num+1, train_loss/(batch_num+1)))

    train_accuracy = correct / len(trainset)

    # Validate
    model.eval()
    num_correct = 0
    for batch_num1, (x, y) in enumerate(testloader):
        x, y = x.to(device), y.to(device)
        outputs = model(x)

        num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()

    val_accuracy = num_correct / len(testset)
    my_acc.append(val_accuracy)
    my_loss.append(train_loss/(batch_num+1))
    print('Epoch: {}\t Training Accuracy: {:.4f}\t Validation Accuracy: {:.4f}\t Avg-Loss: {:.4f}'.format(epoch, train_accuracy*100, val_accuracy * 100, train_loss/(batch_num+1)))
    scheduler.step(val_accuracy)

    #torch.save(network.state_dict(),'/content/drive/MyDrive/DL_CMU/HW2_P2/ResNet_Plateau_d3/Net_'+str(epoch)+'_'+str(val_accuracy)+'_checkpoint.t7')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch: 0	Batch: 1	Avg-Loss: 2.3511
Epoch: 0	Batch: 101	Avg-Loss: 0.9306
Epoch: 0	Batch: 201	Avg-Loss: 0.7690
Epoch: 0	Batch: 301	Avg-Loss: 0.6924
Epoch: 0	Batch: 401	Avg-Loss: 0.6461
Epoch: 0	Batch: 501	Avg-Loss: 0.6004
Epoch: 0	Batch: 601	Avg-Loss: 0.5677
Epoch: 0	Batch: 701	Avg-Loss: 0.5406


RuntimeError: ignored

In [ ]:
# Train!
numEpochs = 100
for epoch in range(numEpochs):
    
    # Train
    model.train()
    train_loss = 0.0
    train_acc = 0.0
    correct = 0

    for batch_num, (x, y) in enumerate(trainloader):
        optimizer.zero_grad()
        
        x, y = x.to(device), y.to(device)

        outputs = model(x)

        correct += (torch.argmax(outputs, axis=1) == y).sum().item()

        loss = criterion(outputs, y.long())
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        if batch_num % 100 == 0:
            print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch, batch_num+1, train_loss/(batch_num+1)))

    train_accuracy = correct / len(trainset)

    # Validate
    model.eval()
    num_correct = 0
    for batch_num1, (x, y) in enumerate(testloader):
        x, y = x.to(device), y.to(device)
        outputs = model(x)

        num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()

    val_accuracy = num_correct / len(testset)
    my_acc.append(val_accuracy)
    my_loss.append(train_loss/(batch_num+1))
    print('Epoch: {}\t Training Accuracy: {:.4f}\t Validation Accuracy: {:.4f}\t Avg-Loss: {:.4f}'.format(epoch, train_accuracy*100, val_accuracy * 100, train_loss/(batch_num+1)))
    scheduler.step(val_accuracy)

    #torch.save(network.state_dict(),'/content/drive/MyDrive/DL_CMU/HW2_P2/ResNet_Plateau_d3/Net_'+str(epoch)+'_'+str(val_accuracy)+'_checkpoint.t7')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch: 0	Batch: 1	Avg-Loss: 2.3927
Epoch: 0	Batch: 101	Avg-Loss: 1.5158
Epoch: 0	Batch: 201	Avg-Loss: 1.2682
Epoch: 0	Batch: 301	Avg-Loss: 1.1363
Epoch: 0	Batch: 401	Avg-Loss: 1.0519
Epoch: 0	Batch: 501	Avg-Loss: 0.9904
Epoch: 0	Batch: 601	Avg-Loss: 0.9412
Epoch: 0	Batch: 701	Avg-Loss: 0.9059
Epoch: 0	 Training Accuracy: 69.3040	 Validation Accuracy: 43.7900	 Avg-Loss: 0.8818
Epoch: 1	Batch: 1	Avg-Loss: 1.0808
Epoch: 1	Batch: 101	Avg-Loss: 0.9907
Epoch: 1	Batch: 201	Avg-Loss: 0.8702
Epoch: 1	Batch: 301	Avg-Loss: 0.8138
Epoch: 1	Batch: 401	Avg-Loss: 0.7680
Epoch: 1	Batch: 501	Avg-Loss: 0.7417
Epoch: 1	Batch: 601	Avg-Loss: 0.7166
Epoch: 1	Batch: 701	Avg-Loss: 0.7043
Epoch: 1	 Training Accuracy: 76.0920	 Validation Accuracy: 81.1500	 Avg-Loss: 0.6935
Epoch: 2	Batch: 1	Avg-Loss: 0.7420
Epoch: 2	Batch: 101	Avg-Loss: 0.5578
Epoch: 2	Batch: 201	Avg-Loss: 0.5364
Epoch: 2	Batch: 301	Avg-Loss: 0.5347
Epoch: 2	Batch: 401	Avg-Loss: 0.5313
Epoch: 2	Batch: 501	Avg-Loss: 0.5248
Epoch: 2	Batch: 601	Av

KeyboardInterrupt: ignored

In [ ]:
# Train!
numEpochs = 100
for epoch in range(numEpochs):
    
    # Train
    model.train()
    train_loss = 0.0
    train_acc = 0.0
    correct = 0

    for batch_num, (x, y) in enumerate(trainloader):
        optimizer.zero_grad()
        
        x, y = x.to(device), y.to(device)

        outputs = model(x)

        correct += (torch.argmax(outputs, axis=1) == y).sum().item()

        loss = criterion(outputs, y.long())
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        if batch_num % 100 == 0:
            print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch, batch_num+1, train_loss/(batch_num+1)))

    train_accuracy = correct / len(trainset)

    # Validate
    model.eval()
    num_correct = 0
    for batch_num1, (x, y) in enumerate(testloader):
        x, y = x.to(device), y.to(device)
        outputs = model(x)

        num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()

    val_accuracy = num_correct / len(testset)
    my_acc.append(val_accuracy)
    my_loss.append(train_loss/(batch_num+1))
    print('Epoch: {}\t Training Accuracy: {:.4f}\t Validation Accuracy: {:.4f}\t Avg-Loss: {:.4f}'.format(epoch, train_accuracy*100, val_accuracy * 100, train_loss/(batch_num+1)))
    scheduler.step(val_accuracy)

    #torch.save(network.state_dict(),'/content/drive/MyDrive/DL_CMU/HW2_P2/ResNet_Plateau_d3/Net_'+str(epoch)+'_'+str(val_accuracy)+'_checkpoint.t7')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch: 0	Batch: 1	Avg-Loss: 2.3778
Epoch: 0	Batch: 101	Avg-Loss: 1.6199
Epoch: 0	Batch: 201	Avg-Loss: 1.4233
Epoch: 0	Batch: 301	Avg-Loss: 1.3190
Epoch: 0	 Training Accuracy: 55.2460	 Validation Accuracy: 62.2100	 Avg-Loss: 1.2627
Epoch: 1	Batch: 1	Avg-Loss: 1.0804
Epoch: 1	Batch: 101	Avg-Loss: 1.1021
Epoch: 1	Batch: 201	Avg-Loss: 1.1522
Epoch: 1	Batch: 301	Avg-Loss: 1.1413
Epoch: 1	 Training Accuracy: 60.9720	 Validation Accuracy: 59.6500	 Avg-Loss: 1.1151
Epoch: 2	Batch: 1	Avg-Loss: 1.1830
Epoch: 2	Batch: 101	Avg-Loss: 1.0432
Epoch: 2	Batch: 201	Avg-Loss: 1.0140
Epoch: 2	Batch: 301	Avg-Loss: 0.9927
Epoch: 2	 Training Accuracy: 65.8860	 Validation Accuracy: 72.8300	 Avg-Loss: 0.9836
Epoch: 3	Batch: 1	Avg-Loss: 0.8010
Epoch: 3	Batch: 101	Avg-Loss: 0.8658
Epoch: 3	Batch: 201	Avg-Loss: 0.8496
Epoch: 3	Batch: 301	Avg-Loss: 0.8410
Epoch: 3	 Training Accuracy: 70.8140	 Validation Accuracy: 77.0000	 Avg-Loss: 0.8269
Epoch: 4	Batch: 1	Avg-Loss: 0.6917
Epoch: 4	Batch: 101	Avg-Loss: 0.8217
Epoc

KeyboardInterrupt: ignored

In [ ]:
# Train!
numEpochs = 100
for epoch in range(numEpochs):
    
    # Train
    model.train()
    train_loss = 0.0
    train_acc = 0.0
    correct = 0

    for batch_num, (x, y) in enumerate(trainloader):
        optimizer.zero_grad()
        
        x, y = x.to(device), y.to(device)

        outputs = model(x)

        correct += (torch.argmax(outputs, axis=1) == y).sum().item()

        loss = criterion(outputs, y.long())
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        if batch_num % 100 == 0:
            print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch, batch_num+1, train_loss/(batch_num+1)))

    train_accuracy = correct / len(trainset)

    # Validate
    model.eval()
    num_correct = 0
    for batch_num1, (x, y) in enumerate(testloader):
        x, y = x.to(device), y.to(device)
        outputs = model(x)

        num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()

    val_accuracy = num_correct / len(testset)
    my_acc.append(val_accuracy)
    my_loss.append(train_loss/(batch_num+1))
    print('Epoch: {}\t Training Accuracy: {:.4f}\t Validation Accuracy: {:.4f}\t Avg-Loss: {:.4f}'.format(epoch, train_accuracy*100, val_accuracy * 100, train_loss/(batch_num+1)))
    scheduler.step(val_accuracy)

    #torch.save(network.state_dict(),'/content/drive/MyDrive/DL_CMU/HW2_P2/ResNet_Plateau_d3/Net_'+str(epoch)+'_'+str(val_accuracy)+'_checkpoint.t7')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch: 0	Batch: 1	Avg-Loss: 2.3666
Epoch: 0	Batch: 101	Avg-Loss: 1.6057
Epoch: 0	Batch: 201	Avg-Loss: 1.3843
Epoch: 0	Batch: 301	Avg-Loss: 1.2689
Epoch: 0	 Training Accuracy: 57.3980	 Validation Accuracy: 71.6800	 Avg-Loss: 1.2038
Epoch: 1	Batch: 1	Avg-Loss: 1.0228
Epoch: 1	Batch: 101	Avg-Loss: 0.9367
Epoch: 1	Batch: 201	Avg-Loss: 0.9324
Epoch: 1	Batch: 301	Avg-Loss: 0.9158
Epoch: 1	 Training Accuracy: 68.6020	 Validation Accuracy: 76.3700	 Avg-Loss: 0.9003
Epoch: 2	Batch: 1	Avg-Loss: 1.0382
Epoch: 2	Batch: 101	Avg-Loss: 0.8229
Epoch: 2	Batch: 201	Avg-Loss: 0.8131
Epoch: 2	Batch: 301	Avg-Loss: 0.8031
Epoch: 2	 Training Accuracy: 72.1500	 Validation Accuracy: 77.3200	 Avg-Loss: 0.7999
Epoch: 3	Batch: 1	Avg-Loss: 0.6936
Epoch: 3	Batch: 101	Avg-Loss: 0.7465
Epoch: 3	Batch: 201	Avg-Loss: 0.7558
Epoch: 3	Batch: 301	Avg-Loss: 0.7535
Epoch: 3	 Training Accuracy: 74.2880	 Validation Accuracy: 79.0800	 Avg-Loss: 0.7465
Epoch: 4	Batch: 1	Avg-Loss: 0.6616
Epoch: 4	Batch: 101	Avg-Loss: 0.6991
Epoc

KeyboardInterrupt: ignored

In [ ]:
# Train!
numEpochs = 100
for epoch in range(numEpochs):
    
    # Train
    model.train()
    train_loss = 0.0
    train_acc = 0.0
    correct = 0

    for batch_num, (x, y) in enumerate(trainloader):
        optimizer.zero_grad()
        
        x, y = x.to(device), y.to(device)

        outputs = model(x)

        correct += (torch.argmax(outputs, axis=1) == y).sum().item()

        loss = criterion(outputs, y.long())
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        if batch_num % 100 == 0:
            print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch, batch_num+1, train_loss/(batch_num+1)))

    train_accuracy = correct / len(trainset)

    # Validate
    model.eval()
    num_correct = 0
    for batch_num1, (x, y) in enumerate(testloader):
        x, y = x.to(device), y.to(device)
        outputs = model(x)

        num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()

    val_accuracy = num_correct / len(testset)
    my_acc.append(val_accuracy)
    my_loss.append(train_loss/(batch_num+1))
    print('Epoch: {}\t Training Accuracy: {:.4f}\t Validation Accuracy: {:.4f}\t Avg-Loss: {:.4f}'.format(epoch, train_accuracy*100, val_accuracy * 100, train_loss/(batch_num+1)))
    scheduler.step(val_accuracy)

    #torch.save(network.state_dict(),'/content/drive/MyDrive/DL_CMU/HW2_P2/ResNet_Plateau_3/Net_'+str(epoch)+'_'+str(val_accuracy)+'_checkpoint.t7')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch: 0	Batch: 1	Avg-Loss: 2.4260
Epoch: 0	Batch: 101	Avg-Loss: 1.9716
Epoch: 0	Batch: 201	Avg-Loss: 1.7083
Epoch: 0	Batch: 301	Avg-Loss: 1.5676
Epoch: 0	 Validation Accuracy: 63.4900	 Avg-Loss: 1.4767
Epoch: 1	Batch: 1	Avg-Loss: 1.0468
Epoch: 1	Batch: 101	Avg-Loss: 1.1158
Epoch: 1	Batch: 201	Avg-Loss: 1.0855
Epoch: 1	Batch: 301	Avg-Loss: 1.0600
Epoch: 1	 Validation Accuracy: 69.8500	 Avg-Loss: 1.0416
Epoch: 2	Batch: 1	Avg-Loss: 0.8544
Epoch: 2	Batch: 101	Avg-Loss: 0.9327
Epoch: 2	Batch: 201	Avg-Loss: 0.9241
Epoch: 2	Batch: 301	Avg-Loss: 0.9190
Epoch: 2	 Validation Accuracy: 73.6200	 Avg-Loss: 0.9112
Epoch: 3	Batch: 1	Avg-Loss: 0.8604
Epoch: 3	Batch: 101	Avg-Loss: 0.8475
Epoch: 3	Batch: 201	Avg-Loss: 0.8353
Epoch: 3	Batch: 301	Avg-Loss: 0.8315
Epoch: 3	 Validation Accuracy: 76.3800	 Avg-Loss: 0.8308
Epoch: 4	Batch: 1	Avg-Loss: 0.7236
Epoch: 4	Batch: 101	Avg-Loss: 0.7743
Epoch: 4	Batch: 201	Avg-Loss: 0.7701
Epoch: 4	Batch: 301	Avg-Loss: 0.7746
Epoch: 4	 Validation Accuracy: 76.5000	 Av

KeyboardInterrupt: ignored

In [ ]:
# Train!
numEpochs = 100
for epoch in range(numEpochs):
    
    # Train
    model.train()
    train_loss = 0.0
    train_acc = 0.0
    correct = 0

    for batch_num, (x, y) in enumerate(trainloader):
        optimizer.zero_grad()
        
        x, y = x.to(device), y.to(device)

        outputs = model(x)

        correct += (torch.argmax(outputs, axis=1) == y).sum().item()

        loss = criterion(outputs, y.long())
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        if batch_num % 100 == 0:
            print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch, batch_num+1, train_loss/(batch_num+1)))

    train_accuracy = correct / len(trainset)

    # Validate
    model.eval()
    num_correct = 0
    for batch_num1, (x, y) in enumerate(testloader):
        x, y = x.to(device), y.to(device)
        outputs = model(x)

        num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()

    val_accuracy = num_correct / len(testset)
    my_acc.append(val_accuracy)
    my_loss.append(train_loss/(batch_num+1))
    print('Epoch: {}\t Training Accuracy: {:.4f}\t Validation Accuracy: {:.4f}\t Avg-Loss: {:.4f}'.format(epoch, train_accuracy*100, val_accuracy * 100, train_loss/(batch_num+1)))
    scheduler.step(val_accuracy)

    #torch.save(network.state_dict(),'/content/drive/MyDrive/DL_CMU/HW2_P2/ResNet_Plateau_d3/Net_'+str(epoch)+'_'+str(val_accuracy)+'_checkpoint.t7')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch: 0	Batch: 1	Avg-Loss: 0.2056
Epoch: 0	Batch: 101	Avg-Loss: 0.2359
Epoch: 0	Batch: 201	Avg-Loss: 0.2357
Epoch: 0	Batch: 301	Avg-Loss: 0.2367
Epoch: 0	 Training Accuracy: 91.8280	 Validation Accuracy: 83.7600	 Avg-Loss: 0.2370
Epoch: 1	Batch: 1	Avg-Loss: 0.2385
Epoch: 1	Batch: 101	Avg-Loss: 0.2445
Epoch: 1	Batch: 201	Avg-Loss: 0.2455
Epoch: 1	Batch: 301	Avg-Loss: 0.2415
Epoch: 1	 Training Accuracy: 91.5680	 Validation Accuracy: 83.5100	 Avg-Loss: 0.2396
Epoch: 2	Batch: 1	Avg-Loss: 0.2614
Epoch: 2	Batch: 101	Avg-Loss: 0.2340
Epoch: 2	Batch: 201	Avg-Loss: 0.2400
Epoch: 2	Batch: 301	Avg-Loss: 0.2385
Epoch: 2	 Training Accuracy: 91.8520	 Validation Accuracy: 83.7900	 Avg-Loss: 0.2362
Epoch: 3	Batch: 1	Avg-Loss: 0.2465
Epoch: 3	Batch: 101	Avg-Loss: 0.2334
Epoch: 3	Batch: 201	Avg-Loss: 0.2387
Epoch: 3	Batch: 301	Avg-Loss: 0.2404
Epoch: 3	 Training Accuracy: 91.7700	 Validation Accuracy: 83.5500	 Avg-Loss: 0.2400
Epoch    46: reducing learning rate of group 0 to 1.0000e-08.
Epoch: 4	Batch

KeyboardInterrupt: ignored

In [ ]:
# Validate
model.eval()
num_correct = 0
for batch_num1, (x, y) in enumerate(trainloader):
    x, y = x.to(device), y.to(device)
    outputs = model(x)

    num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()

val_accuracy = num_correct / len(testset)
my_acc.append(val_accuracy)
my_loss.append(train_loss/(batch_num+1))
print('Epoch: {}\t Train Accuracy: {:.4f}\t Avg-Loss: {:.4f}'.format(epoch, val_accuracy * 100, train_loss/(batch_num+1)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch: 3	 Train Accuracy: 50.0000	 Avg-Loss: 2.2719


In [ ]:
# Validate
model.eval()
num_correct = 0
for batch_num1, (x, y) in enumerate(trainloader):
    x, y = x.to(device), y.to(device)
    outputs = model(x)

    num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()

val_accuracy = num_correct / len(testset)
my_acc.append(val_accuracy)
my_loss.append(train_loss/(batch_num+1))
print('Epoch: {}\t Train Accuracy: {:.4f}\t Avg-Loss: {:.4f}'.format(epoch, val_accuracy * 100, train_loss/(batch_num+1)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch: 11	 Train Accuracy: 50.0000	 Avg-Loss: 2.2609


In [ ]:
# Train!
numEpochs = 40
for epoch in range(numEpochs):
    
    # Train
    model.train()
    total_loss = 0.0
    L_total = 0
    val_total_loss = 0.0
    for batch_num, (x, y, x_len, y_len) in enumerate(train_loader):
        optimizer.zero_grad()
        
        x, y = x.to(device), y.to(device)

        out, out_lens = model(x, x_len)

        loss = criterion(out, y, out_lens, y_len)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if batch_num % 50 == 0:
            print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch, batch_num+1, total_loss/(batch_num+1)))   


    # Validate
    model.eval()
    for batch_num1, (x, y, x_len, y_len) in enumerate(test_loader):
        x, y = x.to(device), y.to(device)
        out, out_lens = model(x, x_len)
        loss = criterion(out, y, out_lens, y_len)
        val_total_loss += loss.item()
        L_total += test_metric(out, out_lens, y, y_len)

    avg_loss, L_avg_dis, val_avg_loss = total_loss/(batch_num+1), L_total/(batch_num1+1), val_total_loss/(batch_num1+1)
    print('Epoch: {}\tAvg-Loss: {:.4f}\tVal-Avg-Loss: {:.4f}\tAvg-Levenstein-dis: {:.4f}'.format(epoch, avg_loss, val_avg_loss, L_avg_dis))
    torch.save(model.state_dict(),'/content/drive/MyDrive/DL_CMU/HW3P2/Conv_3/Net_'+str(epoch)+'_'+str(L_avg_dis)+'_checkpoint.t7')
    scheduler.step(val_avg_loss)